In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from operator import itemgetter

In [2]:
import yaml
with open('../data_code/config.yaml') as f:
    config = yaml.safe_load(f)
    data_path = config['data_path']

In [3]:
icu = pd.read_csv(data_path + 'prep/icustays_MV.csv')
chart = pd.read_pickle(data_path + 'prep/chartevents.p')

In [4]:
chart['ITEMID'] = chart['ITEMID'].replace({
    '223900': 'GCS verbal',
    '223901': 'GCS motor',
    '220739': 'GCS eye',
    '226732': 'O2 delivery device',
})

In [5]:
NEWS_components = [
    'HR', 
    'RR', 
    'SysBP', 
    'Temperature', 
    'SpO2', 
    'GCS verbal',
    'GCS motor',
    'GCS eye',
    'O2 delivery device',
]

In [6]:
items = chart[chart["ITEMID"].isin(NEWS_components)].copy().reset_index(drop=True)

In [7]:
len(items)

13807467

# Convert GCS to AVPU (15: A, 1-14: VPU)

| *        | Eye Opening | Eye Opening | Eye Opening | Eye Opening | Verbal   | Verbal   | Verbal         | Verbal           | Verbal | Motor   | Motor      | Motor      | Motor   | Motor     | Motor |
|----------|-------------|-------------|-------------|-------------|----------|----------|----------------|------------------|--------|---------|------------|------------|---------|-----------|-------|
| Response | Spontaneous | To Speech   | To Pain     | None        | Oriented | Confused | Inapproapriate | Incomprehensible | None   | Obeying | Localizing | Withdrawal | Flexing | Extending | None  |
| Score    | 4           | 3           | 2           | 1           | 5        | 4        | 5              | 2                | 1      | 6       | 5          | 4          | 3       | 2         | 1     |


In [8]:
verbal_mask = (items['ITEMID'] == 'GCS verbal')
items.loc[verbal_mask, 'VALUE'] = items.loc[verbal_mask, 'VALUE'].map({
    "No Response-ETT": 1,
    "No Response": 1,
    "Incomprehensible sounds": 2,
    "Inappropriate Words": 3,
    "Confused": 4,
    "Oriented": 5,
})

motor_mask = (items['ITEMID'] == 'GCS motor')
items.loc[motor_mask, 'VALUE'] = items.loc[motor_mask, 'VALUE'].map({
    "No response": 1,
    "Abnormal extension": 2,
    "Abnormal Flexion": 3,
    "Flex-withdraws": 4,
    "Localizes Pain": 5,
    "Obeys Commands": 6,
})


eye_mask = (items['ITEMID'] == 'GCS eye')
items.loc[eye_mask, 'VALUE'] = items.loc[eye_mask, 'VALUE'].map({
    "None": 1,
    "To Pain": 2,
    "To Speech": 3,
    "Spontaneously": 4,
})

oxygen_mask = (items['ITEMID'] == 'O2 delivery device')
items.loc[oxygen_mask, 'VALUE'] = items.loc[oxygen_mask, 'VALUE'].map({
    'Nasal cannula': 1, 
    'Aerosol-cool': 1, 
    'Bipap mask ': 1, 
    'None': 0,
    'CPAP mask ': 1,
    'Endotracheal tube': 1, 
    'Face tent': 1,
    'Medium conc mask ': 1,
    'Other': 1, 
    'Venti mask ': 1, 
    'Non-rebreather': 1,
    'Tracheostomy tube': 1, 
    'Trach mask ': 1, 
    'High flow neb': 1,
    'High flow nasal cannula': 1, 
    'T-piece': 1, 
    'Vapomist': 1,
    'Ultrasonic neb': 1,
})

In [9]:
assert set(items[items['ITEMID'] == "O2 delivery device"]['VALUE'].unique()) == set([0,1])

In [10]:
items.head(10)

,ICUSTAY_ID,t,ITEMID,VALUE
0,200001,-0.003333,HR,115
1,200001,0.013333,RR,22
2,200001,0.030000,SysBP,113
3,200001,0.130000,SpO2,94
4,200001,0.163333,RR,26
5,200001,0.163333,SpO2,95
6,200001,0.163333,HR,114
7,200001,0.163333,Temperature,37.2778
8,200001,0.196667,O2 delivery device,1
9,200001,0.213333,GCS eye,4


# Map values to bins & Aggregate scores
<img src="NEWS_table.jpg">

In [11]:
HR_bins   = [41, 51, 91, 111, 131]
RR_bins   = [9, 12, 21, 25]
SpO2_bins = [92, 94, 96]
Temp_bins = [35.1, 36.1, 38.1, 39.1]
SBP_bins  = [91, 101, 111, 220]

HR_dict   = { 0:3, 1:1, 2:0, 3:1, 4:2, 5:3 }
RR_dict   = { 0:3, 1:1, 2:0, 3:2, 4:3 }
SpO2_dict = { 0:3, 1:2, 2:1, 3:0 }
Temp_dict = { 0:3, 1:1, 2:0, 3:1, 4:2 }
SBP_dict  = { 0:3, 1:2, 2:1, 3:0, 4:3 }

In [12]:
# test binning
assert SpO2_dict[np.digitize(90, SpO2_bins)] == 3
assert HR_dict[np.digitize(80, HR_bins)] == 0

In [13]:
def calculate_score(row):
    gcs_score = int(row['GCS motor']) + int(row['GCS verbal']) + int(row['GCS eye'])
    score = \
        HR_dict[np.digitize(float(row['HR']), HR_bins)] + \
        RR_dict[np.digitize(float(row['RR']), RR_bins)] + \
        SBP_dict[np.digitize(float(row['SysBP']), SBP_bins)] + \
        Temp_dict[np.digitize(float(row['Temperature']), Temp_bins)] + \
        SpO2_dict[np.digitize(float(row['SpO2']), SpO2_bins)] + \
        2 * int(int(row["O2 delivery device"]) == 1) + \
        3 * int(gcs_score < 15)
    return score

In [14]:
IDs_test = list(icu[icu['partition'] == 'test']["ICUSTAY_ID"])

In [15]:
for T in [4, 12, 48]:
    score_dict = {}
    for ID in tqdm(IDs_test):
        try:
            df = items[items['ICUSTAY_ID'] == ID].sort_values(by='t')
            df = df[df['t'] <= T]
            df = df.drop_duplicates(subset=['ITEMID', 't'], keep='last')
            df = df.pivot(index='t', columns='ITEMID', values='VALUE')
            df = df.reindex(NEWS_components, axis=1)
            df = df.ffill().dropna()
            if len(df) > 0:
                df['NEWS'] = df.apply(calculate_score, axis=1)
                score_dict[ID] = df['NEWS'].max()
            else:
                score_dict[ID] = np.nan
        except:
            print(ID)
            display(df)
            break
    
    df_scores = pd.DataFrame(score_dict.items(), columns=['ID', 'NEWS']).sort_values(by='ID')
    df_scores.to_csv(data_path + 'labels/NEWS_{}h.csv'.format(T), index=False)

100%|██████████| 3492/3492 [04:18<00:00, 13.48it/s]
